In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import IPython.display as ipd
import pandas as pd
import numpy as np
import seaborn as sns
import librosa
import soundfile as sf
import re
import scipy
import torch
import matplotlib.pyplot as plt
import nnAudio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import gc
import seaborn as sns
from datasets import load_dataset

from glob import glob
from tqdm import tqdm
from nnAudio.features.mel import MelSpectrogram
from torchaudio.transforms import AmplitudeToDB
from torch.utils.data import DataLoader, ConcatDataset
from transformers import (
    AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC, 
    AutoProcessor, Wav2Vec2Model, HubertForCTC, 
    HubertModel, Data2VecAudioForCTC, Data2VecAudioModel,
    AutoModelForAudioXVector, Wav2Vec2Processor, Wav2Vec2ForCTC, Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer
)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline
%matplotlib inline

DEVICE = "cuda"

# Dataset

In [2]:
ds = load_dataset('timit_asr', data_dir="../../data/TIMIT")
ds_train, ds_test = ds['train'], ds['test']

c:\Users\famil\AppData\Local\Programs\Python\Python38\lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for timit_asr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/timit_asr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [13]:
ds_train[0]

{'file': 'c:\\DATA\\UCU\\YEAR_3\\AUDIO\\ucu_audio_processing_course\\data\\TIMIT\\data\\lisa\\data\\timit\\raw\\TIMIT\\train\\DR1\\FCJF0\\SA1.WAV',
 'audio': {'path': 'c:\\DATA\\UCU\\YEAR_3\\AUDIO\\ucu_audio_processing_course\\data\\TIMIT\\data\\lisa\\data\\timit\\raw\\TIMIT\\train\\DR1\\FCJF0\\SA1.WAV',
  'array': array([ 3.05175781e-05, -3.05175781e-05,  6.10351562e-05, ...,
         -3.05175781e-05, -1.52587891e-04, -2.44140625e-04]),
  'sampling_rate': 16000},
 'text': 'She had your dark suit in greasy wash water all year.',
 'phonetic_detail': {'start': [0,
   3050,
   4559,
   5723,
   6642,
   8772,
   9190,
   10337,
   11517,
   12500,
   12640,
   14714,
   15870,
   16334,
   18088,
   20417,
   21199,
   22560,
   22920,
   23271,
   24229,
   25566,
   27156,
   28064,
   29660,
   31719,
   33360,
   33754,
   34715,
   36080,
   36326,
   37556,
   39561,
   40313,
   42059,
   43479,
   44586],
  'stop': [3050,
   4559,
   5723,
   6642,
   8772,
   9190,
   10337,
   1

In [3]:
train_phonetics = [phone for x in ds_train for phone in x['phonetic_detail']['utterance']]
print("num of train phones:\t", len(set(train_phonetics)))

test_phonetics = [phone for x in ds_test for phone in x['phonetic_detail']['utterance']]
print("num of train phones:\t", len(set(test_phonetics)))

num of train phones:	 61
num of train phones:	 61


In [4]:
#source: https://www.kaggle.com/code/vitouphy/phoneme-recognition-with-wav2vec2
phon61_map = {
    'iy':'iy',  'ih':'ih',   'eh':'eh',  'ae':'ae',    'ix':'ih',  'ax':'ah',   'ah':'ah',  'uw':'uw',
    'ux':'uw',  'uh':'uh',   'ao':'aa',  'aa':'aa',    'ey':'ey',  'ay':'ay',   'oy':'oy',  'aw':'aw',
    'ow':'ow',  'l':'l',     'el':'l',  'r':'r',      'y':'y',    'w':'w',     'er':'er',  'axr':'er',
    'm':'m',    'em':'m',     'n':'n',    'nx':'n',     'en':'n',  'ng':'ng',   'eng':'ng', 'ch':'ch',
    'jh':'jh',  'dh':'dh',   'b':'b',    'd':'d',      'dx':'dx',  'g':'g',     'p':'p',    't':'t',
    'k':'k',    'z':'z',     'zh':'sh',  'v':'v',      'f':'f',    'th':'th',   's':'s',    'sh':'sh',
    'hh':'hh',  'hv':'hh',   'pcl':'h#', 'tcl':'h#', 'kcl':'h#', 'qcl':'h#','bcl':'h#','dcl':'h#',
    'gcl':'h#','h#':'h#',  '#h':'h#',  'pau':'h#', 'epi': 'h#','nx':'n',   'ax-h':'ah','q':'h#',
        "a": "ə", "ey": "eɪ", "aa": "ɑ", "ae": "æ", "ah": "ə", "ao": "ɔ",
        "aw": "aʊ", "ay": "aɪ", "ch": "ʧ", "dh": "ð", "eh": "ɛ", "er": "ər",
        "hh": "h", "ih": "ɪ", "jh": "ʤ", "ng": "ŋ",  "ow": "oʊ", "oy": "ɔɪ",
        "sh": "ʃ", "th": "θ", "uh": "ʊ", "uw": "u", "zh": "ʒ", "iy": "i", "y": "j"
}

def convert_phon61_to_phon39(sentence):
    # tokens = [phon61_map[x] for x in sentence]
    # return " ".join(tokens)
    return [phon61_map[x] for x in sentence]

def normalize_phones(item):
    item['phonetic_detail']['utterance'] = convert_phon61_to_phon39(item['phonetic_detail']['utterance'])
    return item

In [5]:
ds_train_norm = ds_train.map(normalize_phones)
ds_test_norm = ds_test.map(normalize_phones)

In [6]:
train_phonetics_norm = [phone for x in ds_train_norm for phone in x['phonetic_detail']['utterance']]
print("num of train phones:\t", len(set(train_phonetics_norm)))

test_phonetics_norm = [phone for x in ds_test_norm for phone in x['phonetic_detail']['utterance']]
print("num of test phones:\t", len(set(test_phonetics_norm)))

num of train phones:	 47
num of test phones:	 47


In [7]:
vocab_list = sorted(list(set(train_phonetics_norm + test_phonetics_norm + ["UNK"])))
print(vocab_list)

['UNK', 'ah', 'aɪ', 'aʊ', 'b', 'd', 'dx', 'er', 'eɪ', 'f', 'g', 'h', 'h#', 'hh', 'i', 'ih', 'j', 'k', 'l', 'm', 'n', 'ng', 'oʊ', 'p', 'r', 's', 't', 'u', 'uw', 'v', 'w', 'z', 'æ', 'ð', 'ŋ', 'ɑ', 'ɔ', 'ɔɪ', 'ə', 'ər', 'ɛ', 'ɪ', 'ʃ', 'ʊ', 'ʒ', 'ʤ', 'ʧ', 'θ']


In [8]:
target_label_encoder = LabelEncoder().fit(vocab_list)

In [9]:
target_label_encoder.transform(ds_train_norm[0]["phonetic_detail"]["utterance"])

array([12, 42, 15, 13, 40, 12, 45, 41, 12,  5, 38, 12, 17, 25, 28, 12, 20,
       12, 10, 24, 15, 25, 15, 30, 36, 42, 12, 30, 36,  6,  7, 36, 18, 16,
       41,  7, 12])

In [10]:
train_y = [target_label_encoder.transform(ds_train_norm[i]["phonetic_detail"]["utterance"]) for i in range(len(ds_train_norm))]
test_y = [target_label_encoder.transform(ds_test_norm[i]["phonetic_detail"]["utterance"]) for i in range(len(ds_test_norm))]

In [11]:
def add_labels(entry):
    entry["labels"] = target_label_encoder.transform(entry["phonetic_detail"]["utterance"])
    return entry

In [12]:
ds_train_norm = ds_train_norm.map(add_labels)
ds_test_norm = ds_test_norm.map(add_labels)

In [13]:
ds_train_norm[0]["labels"]

[12,
 42,
 15,
 13,
 40,
 12,
 45,
 41,
 12,
 5,
 38,
 12,
 17,
 25,
 28,
 12,
 20,
 12,
 10,
 24,
 15,
 25,
 15,
 30,
 36,
 42,
 12,
 30,
 36,
 6,
 7,
 36,
 18,
 16,
 41,
 7,
 12]

# Evaluation metric (PER)

In [14]:
def calculate_PER(reference, hypothesis):
    """
    Calculate Phoneme Error Rate (PER) between predicted and true phoneme sequences.    
    """
    n = len(reference)
    m = len(hypothesis)

    D = [[0] * (m + 1) for _ in range(n + 1)]
    
    for i in range(n + 1):
        D[i][0] = i
    for j in range(m + 1):
        D[0][j] = j
    
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if reference[i - 1] == hypothesis[j - 1]:
                D[i][j] = D[i - 1][j - 1]
            else:
                D[i][j] = min(D[i - 1][j] + 1,  # deletion
                              D[i][j - 1] + 1,  # insertion
                              D[i - 1][j - 1] + 1)  # substitution
    
    return D[n][m] / n

In [15]:
def compute_total_PER(predict_list, true_list):
    return np.mean([calculate_PER(predict_list[i], true_list[i]) for i in range(len(predict_list))])

In [19]:
reference_phonemes = [1, 2, 1]
hypothesis_phonemes = [1, 2, 3, 4]
PER = calculate_PER(reference_phonemes, hypothesis_phonemes)
print("Phoneme Error Rate:", PER)

Phoneme Error Rate: 0.6666666666666666


# Wav2Vec

In [16]:
# thanks god it finally works
from phonemizer.backend.espeak.wrapper import EspeakWrapper
_ESPEAK_LIBRARY = 'C:\Program Files\eSpeak NG\libespeak-ng.dll'
EspeakWrapper.set_library(_ESPEAK_LIBRARY)

In [17]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")

Some weights of the model checkpoint at facebook/wav2vec2-lv-60-espeak-cv-ft were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-lv-60-espeak-cv-ft and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably

In [39]:
# tokenize
input_values = processor(ds_train[0]["audio"]["array"], return_tensors="pt", sampling_rate=16000).input_values

with torch.no_grad():
  outputs = model(input_values, output_hidden_states=True)


In [77]:
outputs.hidden_states[-1][0]

tensor([[-0.0347, -0.0542,  0.0501,  ...,  0.0710, -0.0322,  0.0433],
        [-0.0329, -0.0493,  0.0476,  ...,  0.0611, -0.0296,  0.0419],
        [-0.0378, -0.0434,  0.0737,  ...,  0.0549, -0.0293,  0.0383],
        ...,
        [-0.0236, -0.0921, -0.0580,  ...,  0.0870, -0.0522,  0.0427],
        [-0.0248, -0.0752, -0.1010,  ...,  0.1145, -0.0296,  0.0333],
        [-0.0458, -0.0513,  0.0274,  ...,  0.0900, -0.0200,  0.0343]])

In [22]:
logits = outputs.logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

In [23]:
print(ds_train['text'][0])
print(ds_train_norm['phonetic_detail'][0]['utterance'])
print(transcription)

She had your dark suit in greasy wash water all year.
h# ʃ ih hh ɛ h# ʤ ɪ h# d ə h# k s uw h# n h# g r ih s ih w ɔ ʃ h# w ɔ dx er ɔ l j ɪ er h#
['ʃ iː h æ dʒ ɚ d ʌ k s uː t æ n ɡ ɹ iː s i w ɑː ʃ w ɔː ɾ ɚ ɹ ɔː l j ɪɹ']


In [ ]:
transcription_unk = ["UNK" if x not in vocab_list else x for x in transcription[0].split()]
pred_y = target_label_encoder.transform(transcription_unk)

In [168]:
scores = calculate_PER(train_y[0], pred_y)
scores

0.7567567567567568

In [18]:
def add_hidden_state(entry):
    input_values = processor(entry["audio"]["array"], return_tensors="pt", sampling_rate=16000).input_values
    with torch.no_grad():
        outputs = model(input_values, output_hidden_states=True)
    entry["embs"] = outputs.hidden_states[-1][0]
    return entry

In [19]:
ds_train_small = ds_train_norm.select(range(200))
ds_test_small = ds_test_norm.select(range(100))

In [20]:
ds_train_small = ds_train_small.map(add_hidden_state)
ds_test_small = ds_test_small.map(add_hidden_state)

Parameter 'function'=<function add_hidden_state at 0x000002C05E70D3A0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [21]:
train_embs = ds_train_small["embs"]
train_y_small = ds_train_small["labels"]

test_embs = ds_test_small["embs"]
test_y_small = ds_test_small["labels"]

# LogReg

In [22]:
def predict_phonems(ds, sr=16000):
    phonemes = []
    for i in range(len(ds)):
        input_values = processor(ds[i]["audio"]["array"], return_tensors="pt", sampling_rate=sr).input_values
        with torch.no_grad():
            logits = model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)
        transcription_unk = ["UNK" if x not in vocab_list else x for x in transcription[0].split()]
        pred_y = target_label_encoder.transform(transcription_unk)
        phonemes.append(pred_y)
    return phonemes

def pad_phonems(phonems):
    max_length = max(len(sublist) for sublist in phonems)
    X_padded = [list(sublist) + [0] * (max_length - len(sublist)) for sublist in phonems]
    return np.array(X_padded)
    # return torch.cat(X_padded)

def pad_embs(phonems):
    max_length = max(len(sublist) for sublist in phonems)
    X_padded = [list(sublist) + [[0] * 1024] * (max_length - len(sublist)) for sublist in phonems]
    return np.array(X_padded)
    # return torch.cat(X_padded, 0)

In [23]:
train_w2v = pad_embs(train_embs)
train_y_small_pad = pad_phonems(train_y_small)

test_w2v = pad_embs(test_embs)
test_y_small_pad = pad_phonems(test_y_small)

In [219]:
print(train_w2v.shape)
print(train_y_small_pad.shape)

print(test_w2v.shape)
print(test_y_small_pad.shape)

(200, 318, 1024)
(200, 68)
(100, 371, 1024)
(100, 71)


In [24]:
def dublicate_y(arr_y, arr_embs):
    _, b = arr_y.shape
    _, c, _ = arr_embs.shape
    rows_to_repeat = c // b 
    arr_duplicated = np.repeat(arr_y, rows_to_repeat, axis=1)
    extra_zeros = c - arr_duplicated.shape[1]
    arr2_duplicated_pad = np.pad(arr_duplicated, ((0, 0), (0, extra_zeros)), mode='constant', constant_values=0)
    return arr2_duplicated_pad

In [25]:
res = dublicate_y(train_y_small_pad, train_w2v)
res[0]

array([12, 12, 12, 12, 42, 42, 42, 42, 15, 15, 15, 15, 13, 13, 13, 13, 40,
       40, 40, 40, 12, 12, 12, 12, 45, 45, 45, 45, 41, 41, 41, 41, 12, 12,
       12, 12,  5,  5,  5,  5, 38, 38, 38, 38, 12, 12, 12, 12, 17, 17, 17,
       17, 25, 25, 25, 25, 28, 28, 28, 28, 12, 12, 12, 12, 20, 20, 20, 20,
       12, 12, 12, 12, 10, 10, 10, 10, 24, 24, 24, 24, 15, 15, 15, 15, 25,
       25, 25, 25, 15, 15, 15, 15, 30, 30, 30, 30, 36, 36, 36, 36, 42, 42,
       42, 42, 12, 12, 12, 12, 30, 30, 30, 30, 36, 36, 36, 36,  6,  6,  6,
        6,  7,  7,  7,  7, 36, 36, 36, 36, 18, 18, 18, 18, 16, 16, 16, 16,
       41, 41, 41, 41,  7,  7,  7,  7, 12, 12, 12, 12,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [26]:
def transform_embs_and_y(arr_y, arr_embs):
    "Reshape 3d array of last hiddent states and 2d array of phones correspondingly"
    arr_y_dub = dublicate_y(arr_y, arr_embs)
    return arr_embs.reshape(-1, arr_embs.shape[-1]), arr_y_dub.reshape(-1)

In [27]:
train_w2v_ready, train_y_ready = transform_embs_and_y(train_y_small_pad, train_w2v)
test_w2v_ready, test_y_ready = transform_embs_and_y(test_y_small_pad, test_w2v)

In [28]:
classification_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression())
])

In [29]:
classification_pipeline.fit(train_w2v_ready, train_y_ready)

c:\Users\famil\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('scaler', StandardScaler()), ('logreg', LogisticRegression())])

In [205]:
train_w2v2_predict = classification_pipeline.predict(train_w2v_ready)
print("Train PER score: ", calculate_PER(train_w2v2_predict.tolist(), train_y_ready.tolist()))

Train PER score:  0.0863235294117647


In [258]:
test_w2v2_predict = classification_pipeline.predict(test_w2v_ready)
print("Test PER score: ", calculate_PER(test_w2v2_predict.tolist(), test_y_ready.tolist()))

# Neural Network approach

In [42]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLASSES = len(vocab_list)

class PhonemeClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

In [52]:
# Instance of the neural network, loss, optimizer
input_size = 512
hidden_size = 1024
learning_rate = 1e-3

model = PhonemeClassifier(input_size, hidden_size, NUM_CLASSES).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# Training flow
def train(train_loader, test_loader, model, loss_fn, optimizer):

    for epoch in range(10):  # train for 10 epochs
        train_loss = 0
        print("Epoch no:" , epoch)
        for batch_idx, (cur_data, target) in enumerate(tqdm(train_loader)):
            cur_data, target = cur_data.to(device), target.to(device)

            # Compute prediction error
            y_hat = model(cur_data)
            loss = loss_fn(y_hat, target)

            # Apply backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            
        train_loss /= len(train_loader)
        print('Epoch: {}, Training Loss: {:.3f}'.format(epoch, train_loss))


In [54]:
train_loader = DataLoader(ds_train_norm, batch_size=128, shuffle=True)
test_loader = DataLoader(ds_test_norm, batch_size=128,shuffle=False)

c:\Users\famil\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\utils\data\dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
train(train_loader, test_loader,  model, loss_fn, optimizer)